In [1]:
import os


In [2]:
path= r'C:\Users\virar\Downloads\DataScienceTools-master\DataScienceTools-master\Protein-Analysis_of_Covid19-using-BioPython'
os.chdir(path)

In [3]:
import Bio
import nglview as nv
import pandas as pd
from Bio.Seq import Seq
from Bio.PDB import PDBParser
from Bio.Alphabet import generic_dna, generic_rna,generic_protein, IUPAC
from collections import Counter

In [4]:
dir(Bio.Data.CodonTable)

['Alphabet',
 'AmbiguousCodonTable',
 'AmbiguousForwardTable',
 'CodonTable',
 'IUPAC',
 'IUPACData',
 'NCBICodonTable',
 'NCBICodonTableDNA',
 'NCBICodonTableRNA',
 'TranslationError',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'ambiguous_dna_by_id',
 'ambiguous_dna_by_name',
 'ambiguous_generic_by_id',
 'ambiguous_generic_by_name',
 'ambiguous_rna_by_id',
 'ambiguous_rna_by_name',
 'generic_by_id',
 'generic_by_name',
 'list_ambiguous_codons',
 'list_possible_proteins',
 'make_back_table',
 'register_ncbi_table',
 'standard_dna_table',
 'standard_rna_table',
 'unambiguous_dna_by_id',
 'unambiguous_dna_by_name',
 'unambiguous_rna_by_id',
 'unambiguous_rna_by_name']

Reading the FASTA file of SARS-COV-2 and checking the ID with the genbank

In [5]:
from Bio import SeqIO
for i in SeqIO.parse('sequence.fasta','fasta'):
  print(i )

ID: MN908947.3
Name: MN908947.3
Description: MN908947.3 Severe acute respiratory syndrome coronavirus 2 isolate Wuhan-Hu-1, complete genome
Number of features: 0
Seq('ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGATCTCTTGT...AAA', SingleLetterAlphabet())


In [6]:
covid_dna=SeqIO.read('sequence.fasta','fasta').seq
print('The length of the covid-19 dna is', len(covid_dna))
covid_dna

The length of the covid-19 dna is 29903


Seq('ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGATCTCTTGT...AAA', SingleLetterAlphabet())

In [7]:
#transcribing and translating the covd dna
covid_mrna=covid_dna.transcribe()
covid_protein=covid_mrna.translate()
print('Number of proteins after translating the covid dna is', len(covid_protein))

Number of proteins after translating the covid dna is 9967


C:\Users\virar\AppData\Local\Programs\Python\Python38\lib\site-packages\Bio\Seq.py:2745: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  warnings.warn("Partial codon, len(sequence) not a multiple of three. "


In [8]:
#finding out proteins by the spliting te sequence where the stop codon is present and coverting the data into dataframe
covid_aa=covid_protein.split('*')
covid_aa=[str(i) for i in covid_aa]
covid_aa=pd.DataFrame({'Amino Acids':covid_aa})
covid_aa

,Amino Acids
0,IKGLYLPR
1,QTNQLSISCRSVL
2,TNFKICVAVTRLHA
3,CTHAV
4,LITNYCR
...,...
770,SHIAIFNQCVTLGRT
771,KSHHIFTEATRSTIECTVNNARESCLYGRALMCKINFSSAIPM
772,F
773,


In [9]:
covid_aa['Count']=covid_aa['Amino Acids'].str.len()
covid_aa.head()

,Amino Acids,Count
0,IKGLYLPR,8
1,QTNQLSISCRSVL,13
2,TNFKICVAVTRLHA,14
3,CTHAV,5
4,LITNYCR,7


In [10]:
covid_aa.Count.sort_values(ascending=False)

548    2701
694     290
719     123
695      83
718      63
       ... 
98        0
101       0
103       0
105       0
566       0
Name: Count, Length: 775, dtype: int64

In [11]:
#largest aminoacid after translating the mrna from the genome
covid_aa['Amino Acids'][548]

'CTIVFKRVCGVSAARLTPCGTGTSTDVVYRAFDIYNDKVAGFAKFLKTNCCRFQEKDEDDNLIDSYFVVKRHTFSNYQHEETIYNLLKDCPAVAKHDFFKFRIDGDMVPHISRQRLTKYTMADLVYALRHFDEGNCDTLKEILVTYNCCDDDYFNKKDWYDFVENPDILRVYANLGERVRQALLKTVQFCDAMRNAGIVGVLTLDNQDLNGNWYDFGDFIQTTPGSGVPVVDSYYSLLMPILTLTRALTAESHVDTDLTKPYIKWDLLKYDFTEERLKLFDRYFKYWDQTYHPNCVNCLDDRCILHCANFNVLFSTVFPPTSFGPLVRKIFVDGVPFVVSTGYHFRELGVVHNQDVNLHSSRLSFKELLVYAADPAMHAASGNLLLDKRTTCFSVAALTNNVAFQTVKPGNFNKDFYDFAVSKGFFKEGSSVELKHFFFAQDGNAAISDYDYYRYNLPTMCDIRQLLFVVEVVDKYFDCYDGGCINANQVIVNNLDKSAGFPFNKWGKARLYYDSMSYEDQDALFAYTKRNVIPTITQMNLKYAISAKNRARTVAGVSICSTMTNRQFHQKLLKSIAATRGATVVIGTSKFYGGWHNMLKTVYSDVENPHLMGWDYPKCDRAMPNMLRIMASLVLARKHTTCCSLSHRFYRLANECAQVLSEMVMCGGSLYVKPGGTSSGDATTAYANSVFNICQAVTANVNALLSTDGNKIADKYVRNLQHRLYECLYRNRDVDTDFVNEFYAYLRKHFSMMILSDDAVVCFNSTYASQGLVASIKNFKSVLYYQNNVFMSEAKCWTETDLTKGPHEFCSQHTMLVKQGDDYVYLPYPDPSRILGAGCFVDDIVKTDGTLMIERFVSLAIDAYPLTKHPNQEYADVFHLYLQYIRKLHDELTGHMLDMYSVMLTNDNTSRYWEPEFYEAMYTPHTVLQAVGACVLCNSQTSLRCGACIRRPFLCCKCCYDHVISTSHKLVLSVNPYVCNAPGCDVTDVTQLYLGGMSY

In [12]:
#5 most common amino acid
Counter(covid_protein).most_common(5)

[('L', 886), ('S', 810), ('*', 774), ('T', 679), ('C', 635)]

In [13]:
!pip install nglview

You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [14]:
#visualising the 3 D structure
parser= PDBParser()
structure=parser.get_structure('mmdb_6LU7','mmdb_6LU7.pdb')

In [27]:
view = nv.show_biopython(structure)
view.clear_representations()
#view as ball and stick (atom and bond)
view.add_ball_and_stick()

In [28]:
view

NGLWidget()

In [29]:
view.clear_representations()
view.add_cartoon('protein')
view.add_ball_and_stick('not protein')
view

NGLWidget(n_components=1)